<a href="https://colab.research.google.com/github/fvillenave/test/blob/main/Copie_de_Day2_teacher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Information Retrieval
## Introduction to search engines

### DAY 2: Teacher version
### Implementing a search engine

The goal of this second session is to implement a first architecture of a search engine on the previously introduced dataset (stackexchange-datascience). If you missed the first session or if you did not saved the dataset, please reload the first session's notebook to download it. 

If you need some ifnormation about the dataset, it should be available here : https://archive.org/details/stackexchange

The notebook is divided into several steps:
-	Implement the indexation
-	Implement the search method
-	Define a ranking strategy and implement it
-	Suggest some improvements of the search engine



## Initialisation

In [2]:
!pip install ttable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.3/122.3 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ttable: filename=ttable-0.6.4-cp310-cp310-linux_x86_64.whl size=212629 sha256=1a3455139acffe92b08f9d6f6d0e50d0a9473a7046ca0e0a46f3e4b1e3079d1a
  Stored in directory: /root/.cache/pip/wheels/0d/8d/56/f2572fdbf1ef1f8a947d7ff25ce18d9373d8e02a68f9ac8de6
Successfully built ttable


In [2]:
import pandas as pd
import re
import os
import math
import numpy as np
from sklearn.metrics import mean_squared_error
from tt import BooleanExpression
from itertools import product

In [3]:
# Only if you use Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# TODO:
DATA_PATH = '' 

# CORR:
DATA_PATH = '/content/drive/MyDrive/TP Centrale/data'

# Implement the indexation
As you might already know, for a search engine to work properly an index of the documents must be created. Here we will keep it in python, and try to use only common libraries to keep it simple.

Once created, the index will be used to match the query with the documents. As a result, there are several ways to build an index, using statistical, boolean, semantic indexation...

First of, let's make a naive one that will consist in breaking down each document into a set of the words it contains.

In [5]:
def extract_words(text:str)->list:
  # TODO

  # CORR:
  return list(set(text.split()))

In [6]:

# test
s = "The cat is sat on the mat. The dog is laid on the mat."
assert extract_words(s).sort()==["The","cat","is","sat","on","the","mat.","dog","laid"].sort()

As you may notice, there are several problems with the previous implementation. First, "The" and "the" aren't considered the same, the "." is kept at the the end of "mat." as any other punctuation character... 

Re-implement this function with some basic preprocessing to avoid these issues.

In [7]:
import re

def extract_words(text:str)->list:
  #TODO

  #CORR
  return list(set(re.sub(r'[^\w\s]', '', text.lower()).split()))

In [8]:
# test
assert extract_words(s).sort()==["the","cat","is","sat","on","mat","dog","laid"].sort()

Now you sould be able to create your index table. For now we will just make a dataframe with two columns: [raw_text, words].

In [9]:
import pandas as pd

def index_docs(docs:list[str])->pd.DataFrame:
  df = pd.DataFrame({"raw_docs":docs})
  df['words'] = df['raw_docs'].apply(extract_words)
  return df

In [10]:
# test

L = [s, "Hello World!", "Goodbye", "How are you?"]

index_docs(L)

,raw_docs,words
0,The cat is sat on the mat. The dog is laid on ...,"[on, cat, laid, the, sat, dog, mat, is]"
1,Hello World!,"[world, hello]"
2,Goodbye,[goodbye]
3,How are you?,"[are, you, how]"


Now, let's try it on the dataset:

In [11]:
# Only if you use Colab
from google.colab import drive
drive.mount('/content/drive')

import os

# TODO:
DATA_PATH = '' 

# CORR:
DATA_PATH = '/content/drive/MyDrive/TP Centrale/data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
posts = pd.read_xml(os.path.join(DATA_PATH, 'Posts.xml'), parser="etree", encoding="utf8")
posts

,Id,PostTypeId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastActivityDate,Title,Tags,...,ClosedDate,ContentLicense,AcceptedAnswerId,LastEditorUserId,LastEditDate,ParentId,OwnerDisplayName,CommunityOwnedDate,LastEditorDisplayName,FavoriteCount
0,5,1,2014-05-13T23:58:30.457,9,898.0,<p>I've always been interested in machine lear...,5.0,2014-05-14T00:36:31.077,How can I do simple machine learning without h...,<machine-learning>,...,2014-05-14T14:40:25.950,CC BY-SA 3.0,NaN,NaN,None,NaN,None,None,None,NaN
1,7,1,2014-05-14T00:11:06.457,4,478.0,"<p>As a researcher and instructor, I'm looking...",36.0,2014-05-16T13:45:00.237,What open-source books (or other materials) pr...,<education><open-source>,...,2014-05-14T08:40:54.950,CC BY-SA 3.0,10.0,97.0,2014-05-16T13:45:00.237,NaN,None,None,None,NaN
2,9,2,2014-05-14T00:36:31.077,5,NaN,"<p>Not sure if this fits the scope of this SE,...",51.0,2014-05-14T00:36:31.077,None,None,...,None,CC BY-SA 3.0,NaN,NaN,None,5.0,None,None,None,NaN
3,10,2,2014-05-14T00:53:43.273,13,NaN,"<p>One book that's freely available is ""The El...",22.0,2014-05-14T00:53:43.273,None,None,...,None,CC BY-SA 3.0,NaN,NaN,None,7.0,None,None,None,NaN
4,14,1,2014-05-14T01:25:59.677,26,1901.0,<p>I am sure data science as will be discussed...,66.0,2020-08-16T13:01:33.543,Is Data Science the Same as Data Mining?,<data-mining><definitions>,...,None,CC BY-SA 3.0,29.0,322.0,2014-06-17T16:17:20.473,NaN,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75722,119962,1,2023-03-04T20:06:06.820,0,8.0,<p>I am implementing a neural network of arbit...,147597.0,2023-03-04T20:22:12.523,Back Propagation on arbitrary depth network wi...,<neural-network><backpropagation>,...,None,CC BY-SA 4.0,NaN,147597.0,2023-03-04T20:22:12.523,NaN,None,None,None,NaN
75723,119963,1,2023-03-04T20:12:19.677,0,10.0,<p>I am using KNN for a regression task</p>\n<...,147598.0,2023-03-04T20:12:19.677,Evaluation parameter in knn,<regression><k-nn>,...,None,CC BY-SA 4.0,NaN,NaN,None,NaN,None,None,None,NaN
75724,119964,1,2023-03-05T00:14:12.597,0,7.0,<p>I have developed a small encoding algorithm...,44581.0,2023-03-05T00:14:12.597,Can I use zero-padded input and output layers ...,<deep-learning><convolutional-neural-network>,...,None,CC BY-SA 4.0,NaN,NaN,None,NaN,None,None,None,NaN
75725,119965,1,2023-03-05T00:43:12.213,0,5.0,"<p>To my understanding, optimizing a model wit...",84437.0,2023-03-05T00:43:12.213,Why does cross validation and hyperparameter t...,<cross-validation><hyperparameter-tuning>,...,None,CC BY-SA 4.0,NaN,NaN,None,NaN,None,None,None,NaN


For our first version of the indexation mechanism, we will simply use the "body" of the posts. To have a better search engine, the title and other metadata aswell could be used aswell. Finally, not all the XML files have a "body" feature, so for the search engine to retrieve information from any of the files you will need to implement another way to index.

But first, let's start with "body". There is more to preprocess than before, indeed, there are html tags such as "<p>" for instance. They are not useful for us, because users won't use them in their queries. So we first need to remove them.

In [13]:
def remove_tags(text:str)->str:
  pattern_html = re.compile('<.*?>')
  clean_post = pattern_html.sub('', text)

  # remove other things

  clean_post = clean_post.replace('\n', '').replace('\t', '')
  return clean_post

In [14]:
# test
remove_tags('<p>Hello World!\nI am making a search engine.<p>')

'Hello World!I am making a search engine.'

In [15]:
clean_posts = posts[['Id','Body']]
clean_posts['Clean Body'] = clean_posts['Body'].fillna('').apply(remove_tags)
clean_posts

<ipython-input-15-2f3e0e9b59ce>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_posts['Clean Body'] = clean_posts['Body'].fillna('').apply(remove_tags)


,Id,Body,Clean Body
0,5,<p>I've always been interested in machine lear...,I've always been interested in machine learnin...
1,7,"<p>As a researcher and instructor, I'm looking...","As a researcher and instructor, I'm looking fo..."
2,9,"<p>Not sure if this fits the scope of this SE,...","Not sure if this fits the scope of this SE, bu..."
3,10,"<p>One book that's freely available is ""The El...","One book that's freely available is ""The Eleme..."
4,14,<p>I am sure data science as will be discussed...,I am sure data science as will be discussed in...
...,...,...,...
75722,119962,<p>I am implementing a neural network of arbit...,I am implementing a neural network of arbitrar...
75723,119963,<p>I am using KNN for a regression task</p>\n<...,I am using KNN for a regression taskIt's like ...
75724,119964,<p>I have developed a small encoding algorithm...,I have developed a small encoding algorithm th...
75725,119965,"<p>To my understanding, optimizing a model wit...","To my understanding, optimizing a model with k..."


In [16]:
clean_posts['words'] = clean_posts['Clean Body'].apply(extract_words)
clean_posts

<ipython-input-16-921669650514>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_posts['words'] = clean_posts['Clean Body'].apply(extract_words)


,Id,Body,Clean Body,words
0,5,<p>I've always been interested in machine lear...,I've always been interested in machine learnin...,"[generating, learning, use, placed, dont, of, ..."
1,7,"<p>As a researcher and instructor, I'm looking...","As a researcher and instructor, I'm looking fo...","[relatively, looking, of, in, suitable, a, to,..."
2,9,"<p>Not sure if this fits the scope of this SE,...","Not sure if this fits the scope of this SE, bu...","[it, parameters, its, a, specific, perhaps, th..."
3,10,"<p>One book that's freely available is ""The El...","One book that's freely available is ""The Eleme...","[book, thinking, learning, although, prof, it,..."
4,14,<p>I am sure data science as will be discussed...,I am sure data science as will be discussed in...,"[in, a, to, need, regards, discussed, few, thi..."
...,...,...,...,...
75722,119962,<p>I am implementing a neural network of arbit...,I am implementing a neural network of arbitrar...,"[depth, each, forwards, think, neural, number,..."
75723,119963,<p>I am using KNN for a regression task</p>\n<...,I am using KNN for a regression taskIt's like ...,"[it, sum_of_featur3_normaln, neighbor5, estima..."
75724,119964,<p>I have developed a small encoding algorithm...,I have developed a small encoding algorithm th...,"[seriesi, sense, associated, it, adjacent, its..."
75725,119965,"<p>To my understanding, optimizing a model wit...","To my understanding, optimizing a model with k...","[tuning, metrics, a, reconciled, means, perfor..."


## Zipf Law

A way of analyzing a corpus is to draw the zipf law

In [17]:
# TODO : Draw Zipf Law on the Posts Corpus

## Inverted Index

Now, we want to go further on the indexing and build an inverted index. Inverted index is a dictionary where the keys are the words of the vocabulary and the values are the documents containing these words. Reducing the size of the vocabulary is a relevant first step when building an inverted index. Here, we will focus on the creation of the index, we leave you the optimisation steps :)

In [18]:
def create_index(posts:pd.DataFrame)-> set:
  inverted_index = dict()
  for i in range(0, len(posts)):
    terms = posts['words'].iloc[i]
    for term in terms:
      if term in inverted_index:
        inverted_index[term] = inverted_index[term] + [posts['Id'].iloc[i]]
      else: 
        inverted_index[term] = [posts['Id'].iloc[i]]
  return inverted_index

In [19]:
inverted_index = create_index(clean_posts.iloc[0:5000])

#### Well Done, you've indexed the dataset! 
Don't hesitate to save your indexes in txt or pickle file

---
# Implement the search method

A naive method would be to count the number of words in common between the query and each posts. Then to rank the posts you could directly select the post who maximize the number of common words. Let's implement this approach :

In [20]:
import numpy as np

In [21]:
# Implement the word_in_index function 
# Inputs : a word (str) & a list of words
# Output : pandas series of 1 if the word is in the list, else 0

def word_in_index(word, word_list_index):
  # TODO

  # CORR
  return int(word in word_list_index)

In [22]:
# Implement the function which run through a pandas series and count the number of word in common
# Use extract_words method, apply method with word_in_index function
# Inputs : the query (str) & pandas series of strings
# Output : Pandas series counting the number of common words between the query and each string in word_serie

def count_common_words(query, word_serie):
  # TODO

  # CORR
  word_list_query = extract_words(query)
  common_word_serie = word_serie.apply(lambda row: np.sum([word_in_index(word, row) for word in word_list_query]))
  return common_word_serie


In [23]:

# df : ("words",  "Id") à minima

def rank_top_query(query, df, top=5):
  # TODO


  # CORR
  df["query_rank"] = count_common_words(query, df["words"])

  return df.sort_values(by="query_rank", ascending=False).head(top)

In [24]:
rank_top_query(query="testing the query in python", df=clean_posts, top=5)

<ipython-input-23-487344b0b33e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["query_rank"] = count_common_words(query, df["words"])


,Id,Body,Clean Body,words,query_rank
273,309,<p><strong>tl;dr:</strong> They markedly diffe...,tl;dr: They markedly differ in many aspects an...,"[efficient, it, its, testing, here, characteri...",5
178,205,"<p>Working on what could often be called ""medi...","Working on what could often be called ""medium ...","[it, its, testing, rule, apply, a, anywhere, r...",4
35201,54542,<p>I am pretty new to Python and this board so...,I am pretty new to Python and this board so I ...,"[use, it, testing, classificator, please, a, p...",4
46350,71759,<p>I wanted to ask you which coding language I...,I wanted to ask you which coding language I sh...,"[quotyesquot, use, enter, recurring, he, x, cl...",4
51485,80165,<p>I was trying to implement Logistic Regressi...,I was trying to implement Logistic Regression ...,"[npimport, 3900, 74350233000888215epoch, manda...",4


Testez plusieurs requêtes et critiquez les résultats obtenus.

Quels sont les pros and cons de cette méthodes. Vous l'indiquerez sur le rapport avec vos réflexions pour l'améliorer.

Next, you have to implement the first improvements you find in the search method to get most relevant results 

In [25]:
# CORR
def remove_stop_words(l_txt: list) -> list:
  
  # Implement without stop words, punctuation, decrease the importance of the ranking with respect to the number of words in the posts

  return 

## Boolean Search

Thanks to the ttable library, implement a boolean search method

In [66]:
b = BooleanExpression('data impl not (science nand learning)')

b = BooleanExpression('data AND not science')

from functools import reduce


In [91]:
"data" in clean_posts.iloc[5591].words

False

In [92]:
"science" in clean_posts.iloc[5591].words

False

In [93]:
ZZ[0].iloc[5591]

False

In [72]:
Z = []

df = pd.DataFrame(b.sat_all())
cols = df.columns

for r in df.iterrows():
  irow, row = r

  ZZ = []

  for col in cols:
    print(col, row[col])
    ZZ.append(clean_posts["words"].apply(lambda words:(col in words) == row[col]))

  Z.append(reduce(lambda x, y: x&y, ZZ))

  break

data True
science False


In [99]:
z

0        False
1        False
2        False
3        False
4        False
         ...  
75722    False
75723     True
75724    False
75725     True
75726    False
Name: words, Length: 75727, dtype: bool

In [100]:
z[5591]

False

In [106]:
clean_posts[z]

,Id,Body,Clean Body,words,query_rank
6,16,"<p>I use <a href=""http://www.csie.ntu.edu.tw/~...",I use Libsvm to train data and predict classif...,"[problemlast, liblinear, use, it, mapreduce, l...",1
9,19,<p>Lots of people use the term <em>big data</e...,Lots of people use the term big data in a rath...,"[associated, use, commercial, efficiency, term...",2
10,20,<p>We created a social network application for...,We created a social network application for eL...,"[each, we, it, its, experimental, other, some,...",2
11,21,"<p>As you rightly note, these days ""big data"" ...","As you rightly note, these days ""big data"" is ...","[sense, associated, use, it, its, generous, a,...",2
12,22,<p>My data set contains a number of numeric at...,My data set contains a number of numeric attri...,"[takes, categoricalattrvalue2, categoricalattr...",1
...,...,...,...,...,...
75707,119946,<p>Good morning everyone.</p>\n<p>I have the f...,Good morning everyone.I have the following dat...,"[pandas, 91, 41, 41i, 283, 28, it, 65, 35, 333...",2
75711,119951,<p>I need to implement classical perceptron al...,I need to implement classical perceptron algor...,"[pandas, testing, it, 60k, should, a, mean, 33...",3
75718,119958,<p>I am trying to write a thesis on oil pipe l...,I am trying to write a thesis on oil pipe leak...,"[location, parameters, it, should, a, specific...",2
75723,119963,<p>I am using KNN for a regression task</p>\n<...,I am using KNN for a regression taskIt's like ...,"[it, sum_of_featur3_normaln, neighbor5, estima...",2


In [101]:
post_query = []
for z in Z:
  post_query.extend(clean_posts[z]["Id"].values.tolist())

In [102]:
list(set(post_query))

[16,
 19,
 20,
 21,
 22,
 24,
 25,
 26,
 30,
 33,
 35,
 37,
 38,
 40,
 41,
 42,
 43,
 44,
 45,
 47,
 57,
 59,
 61,
 62,
 64,
 66,
 67,
 70,
 71,
 72,
 74,
 75,
 76,
 77,
 79,
 80,
 83,
 84,
 85,
 86,
 87,
 89,
 90,
 91,
 92,
 93,
 96,
 106,
 107,
 109,
 111,
 116,
 118,
 122,
 124,
 126,
 133,
 134,
 136,
 137,
 138,
 143,
 144,
 145,
 146,
 151,
 152,
 153,
 156,
 157,
 161,
 162,
 163,
 165,
 169,
 174,
 175,
 176,
 177,
 180,
 181,
 183,
 186,
 191,
 194,
 196,
 202,
 204,
 205,
 206,
 207,
 208,
 209,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 222,
 223,
 227,
 228,
 229,
 237,
 241,
 242,
 246,
 247,
 249,
 250,
 251,
 254,
 255,
 257,
 258,
 259,
 264,
 268,
 274,
 275,
 276,
 279,
 280,
 282,
 285,
 289,
 291,
 293,
 294,
 296,
 300,
 301,
 302,
 305,
 306,
 307,
 308,
 309,
 311,
 312,
 314,
 316,
 317,
 318,
 319,
 324,
 325,
 327,
 331,
 337,
 350,
 352,
 356,
 358,
 359,
 360,
 361,
 362,
 363,
 366,
 372,
 373,
 375,
 376,
 377,
 379,
 382,
 384,
 388,
 391,
 394,
 395,
 39

In [90]:
"data" in clean_posts.iloc[5591]["words"]

False

## Probabilistic search

Implement the MIB or BM25 method of searching

Compare the naive method with your improvements and the boolean and probabilistic search. (report)



---



---




# Evaluate the Search

Now you implement multiple search methods and you're able to improve it. You have to define metric to compare it objectively.

We ask you to implement NDCG (Normalized Discounted Cumulative Gain) from few queries we implement on a dozen of post. We already defined the values of relevance judgement in the csv file : . The final score will be the mean quadratic error of the queries.

The scaling value (Z) must be scale to 1.

You will have to criticize this metric and your result in the report. Then you will have to propose some improvements. 

Thereafter in this week, you will have to compare your different search engines.

In [ ]:
# FAKE EXAMPLE

df = pd.DataFrame(columns=["postId", "query", "query_output"], data=[[1, 3, 1], [2, 1, 2], [3, None, 3], [4, 2, 4]])
# df = pd.DataFrame(columns=["postId", "query", "query_output"], data=[[1, 3, 3], [2, 1, 1], [3, None, 4], [4, 2, 2]])

# 

ndgc = []

# For a query

def calculate_ndgc(df, query_col="query", output_col="query_output"):
  nb_post_relevant = df["query"].count()
  v = 0
  df = df.sort_values(by=query_col)
  for irow, row in enumerate(df[output_col]):
    if irow < nb_post_relevant:
      v += (2**(row) - 1) / math.log(irow + 1 + 1, 2)
  return v

# Squared Error between best ndgc and ndgc of the search engine
mean_squared_error([calculate_ndgc(df=df, query_col=q, output_col=q+"_output") for q in ['query']],
                   [calculate_ndgc(df=df, query_col=q, output_col=q) for q in ['query']]
                          )

43.18010488189557

In [ ]:
[calculate_ndgc(df=df, query_col=q, output_col=q+"_output") for q in ['query']], [calculate_ndgc(df=df, query_col=q, output_col=q) for q in ['query']]

([6.392789260714372], [6.392789260714372])

14.9165082750002

In [ ]:
math.avg

AttributeError: ignored

In [ ]:
ndgc

[6.392789260714372]